# Part 1

In [1]:
import pandas as pd
import numpy as np
import requests as req

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html = req.get(url).content
df_list = pd.read_html(html)
my_df = df_list[0]
df = my_df.loc[my_df['Borough'] != 'Not assigned']
df.reset_index(drop = True,inplace = True)

In [3]:
df.loc[df['Neighborhood'] == 'Not assigned' ]

,Postal Code,Borough,Neighborhood


# Part 2

In [4]:
df_coord = pd.read_csv('Geospatial_Coordinates.csv')

In [5]:
res_df = df.merge(df_coord, on = ['Postal Code'])
res_df

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


# Part 3

In [6]:
import folium
from geopy.geocoders import Nominatim

In [7]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 43.6534817, -79.3839347.


In [8]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(res_df['Latitude'], res_df['Longitude'], res_df['Borough'], res_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [9]:
res_df['Borough'].unique()

array(['North York', 'Downtown Toronto', 'Etobicoke', 'Scarborough',
       'East York', 'York', 'East Toronto', 'West Toronto',
       'Central Toronto', 'Mississauga'], dtype=object)

In [17]:
fin_df = res_df[res_df['Borough'].isin(['Downtown Toronto','East Toronto','West Toronto','Central Toronto'])]
fin_df.reset_index(drop = True,inplace = True)
fin_df.shape

(39, 5)

In [15]:
from sklearn.cluster import KMeans

In [19]:
Model = KMeans(n_clusters = 4,random_state = 0).fit(fin_df[['Latitude','Longitude']])

In [21]:
Model.cluster_centers_

array([[ 43.66943648, -79.32465436],
       [ 43.65506566, -79.44547176],
       [ 43.70563855, -79.39811351],
       [ 43.6547639 , -79.38308287]])

In [26]:
# create map
import matplotlib.cm as cm
import matplotlib.colors as colors
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(5)
ys = [i + x + (i*x)**2 for i in range(5)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(fin_df['Latitude'], fin_df['Longitude'], fin_df['Neighborhood'], Model.labels_):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters